# ch 18. Naive Bayes Classifier

naive bayes classifier는 베이즈 정리를 사용한 가장 기본적인 텍스트 분류 기법입니다. 이번 챕터에서는 naive bayes의 기본 동작 원리를 살펴보고, 네이버 영화 리뷰 데이터 셋을 이용하여 직접 텍스트 분류 모델을 학습시켜 보겠습니다.

In [1]:
## 구글 스팸 메일 분류하는 데 사용한 초기 모델이었음
## 데이터가 쌓일수록 정확도가 갱신됨

## 베이즈 정리 복습

베이즈 정리는 사전 확률과 사후 확률의 관계에 대한 정리라고 하였습니다. 새로운 증거를 확보하여 사전 확률을 더 신뢰도있게 갱신시키는 것이 베이즈 정리의 핵심이었습니다.

$$P(H|E)=\frac{P(E|H)P(H)}{P(E)}$$

$$\text{H: Hypothesis, 어떤 사건이 발생했다는 주장}$$

$$\text{E: Evidence, 새로운 정보}$$

$$P(H):\text{어떤 사건이 발생했다는 주장에 대한 신뢰도, 사전 확률}$$

$$P(H|E):\text{새로운 정보를 받은 후 갱신된 신뢰도, 사후 확률}$$

## Naive Bayes Classifier

Naive Bayes는 베이즈 정리를 이용해서 데이터를 분류하는 모델입니다. 여기서는 텍스트의 종류를 분류하는데 적용해보겠습니다.

먼저 영화 리뷰 텍스트를 보고, 긍정적인 리뷰인지, 부정적인 리뷰인지 분류하고 싶다고 가정하겠습니다. 즉, 영화 리뷰가 주어졌을 때, 이 리뷰가 긍정적인 리뷰일 확률이 궁금합니다. 먼저 하나의 리뷰는 document, 하나의 document는 토큰들의 시퀀스로 표현하겠습니다. 

$$d=(x_{1}, x_{2}, ..., x_{n})$$

우리가 구하고 싶은 건 리뷰 텍스트가 주어졌을 때, 이 리뷰가 긍정적인 리뷰일 확률입니다. 베이즈 정리를 이용해서 표현하면 아래와 같습니다.


$$P(pos|d)=\frac{P(d|pos)P(pos)}{P(d)}=\frac{P(d|pos)P(pos)}{P(d|pos)P(pos)+P(d|neg)P(neg)}$$


In [2]:
## p(d)  특정한 도큐먼트가 등장할 확률  -> 쪼개보면 -> 긍정적, 부정적 리뷰 비율 
## (긍정적 리뷰 중에서 d(토큰)이 등장할 확률)  /  (긍정적 리뷰 중에서 d(토큰)이 등장할 확률) + (부정적 리뷰 중에서 d(토큰)이 등장할 확률)
## 알고 싶은 건 리뷰가 하나 주어졌을 때 긍정인지 부정인지 알고 싶은 거임

여기서 P(pos)와 P(neg)는 사전 확률로 이미 우리가 알고 있다고 가정하겠습니다. 문서가 주어졌을 때, 이 문서가 긍정일 확률은 사후 확률에 해당합니다. 이를 알기 위해서는 P(d|pos)와 P(d|neg)를 계산해야합니다. 이를 몇가지 가정을 통해 계산하는 모델이 Naive Bayes 입니다.

In [ ]:
## 데이터 0 부정, 1 긍정

In [3]:
## P(d|pos) : 긍정적일 때 특정 리뷰가 등장할 확률

### Naive Bays 가정
naive bayes 몇 가지 가정을 통해 아주 간단하게 조건부 확률을 계산합니다.  

1. 문서 내에서 특정 토큰이 등장하는 위치는 신경쓰지 않는다. 오로지 등장 여부만 신경쓴다.(Bag of Words assumption)
2. 문서 내에 특정 토큰이 등장하는 사건은 서로 독립이다.(Conditional Independence) 토큰 간에 연관성은 신경쓰지 않겠다. 

이 가정을 적용하면 아래와 같은 수식을 작성할 수 있습니다.  

$$P(d|pos)$$

$$=P(x_{1},x_{2},...,x_{n}|pos)$$

$$=P(x_{1} \cap x_{2} \cap ... \cap x_{n}|pos)$$

$$=P(x_{1}|pos)*P(x_{2}|pos)*...*P(x_{n}|pos)$$

모든 사건들이 독립이라는 것을 가정하기 때문에 이름에 Naive(순진한)이 붙은 겁니다. 이를 이용하여 사후 확률 값을 계산하고, 사후 확률이 가장 높은 클래스를 선택하는 것이 naive bayes classifier입니다. 이를 수식으로 나타내면 아래와 같습니다.

$$\hat{y}=\underset{y}{argmax}P(y)\prod_{i=1}^{n}P(x_{i}|y)$$

## 예제

### 데이터 셋 준비
실제 데이터를 살펴보면서 naive bayes 알고리즘을 이해해보겠습니다. 아래와 같이 데이터 셋이 주어졌다고 해보겠습니다.

In [8]:
data = [
    ("This movie was awesome! I really enjoyed it.", 1),
    ("This movie is just a masterpiece.", 1),
    ("This movie was boring and waste of time.", 0),
    ("shit, it sucks. I was almost sleeping", 0),
]

각 문장을 토큰화 하고, 각 단어별로 긍정일 때 등장할 조건부 확률, 부정일 때 등장할 조건부 확률을 집계해보겠습니다.

In [4]:
from nltk.tokenize import WordPunctTokenizer
from collections import Counter

tokenizer = WordPunctTokenizer()
pos_counter = Counter()  # 긍정일 때
neg_counter = Counter()  # 부정일 때

In [9]:
for review, label in data:
    tokens = tokenizer.tokenize(review.lower())
    if label == 0:
        neg_counter += Counter(tokens)
    else:
        pos_counter += Counter(tokens)

In [10]:
  print(pos_counter)

Counter({'this': 2, 'movie': 2, '.': 2, 'was': 1, 'awesome': 1, '!': 1, 'i': 1, 'really': 1, 'enjoyed': 1, 'it': 1, 'is': 1, 'just': 1, 'a': 1, 'masterpiece': 1})


In [11]:
  print(neg_counter)

Counter({'was': 2, '.': 2, 'this': 1, 'movie': 1, 'boring': 1, 'and': 1, 'waste': 1, 'of': 1, 'time': 1, 'shit': 1, ',': 1, 'it': 1, 'sucks': 1, 'i': 1, 'almost': 1, 'sleeping': 1})


In [12]:
pos_total_cnt = sum(pos_counter.values())
pos_vocabs = len(pos_counter.keys())        # 유니크한 토큰 개수
neg_total_cnt = sum(neg_counter.values())
neg_vocabs = len(neg_counter.keys())

In [13]:
pos_total_cnt, pos_vocabs

(17, 14)

In [14]:
neg_total_cnt, neg_vocabs

(18, 16)

### 사전 확률
리뷰가 긍정일 사전확률과 부정일 사전 확률은 아래와 같습니다.

$$P(pos)=0.5$$

$$P(neg)=0.5$$

### 조건부 확률 계산
긍정일 때 특정 단어가 등장할 확률은 아래와 같습니다. 부정일 때는 positive를 negative로만 바꿔주면 됩니다. 

한번도 등장하지 않은 단어가 주어졌을 때, 확률값이 0이 되어 전체 확률을 0으로 만들어버리는 현상을 방지하기 위해 분자에 1을 더해주고, 분모에는 전체 어휘의 수를 더해줍니다. 마치 모든 단어를 포함하는 문서가 하나 추가되었다고 생각하면 됩니다. 이러한 기법을 additive smoothing 혹은 laplace smoothing이라고 부릅니다.

additive smoothing: https://en.wikipedia.org/wiki/Additive_smoothing

$$P(w|pos)=\frac{count(w, pos)+1}{\sum_{w \in V}count(w, pos)+V}$$

In [15]:
  print(pos_counter)

Counter({'this': 2, 'movie': 2, '.': 2, 'was': 1, 'awesome': 1, '!': 1, 'i': 1, 'really': 1, 'enjoyed': 1, 'it': 1, 'is': 1, 'just': 1, 'a': 1, 'masterpiece': 1})


In [ ]:
## 분자에 1 더해주고 (새롬게 등장한 토큰이 있으면 0이 나올 수 있는데 이러면 수식에 모든 걸 곱하기 할 때 0이 될 수 있음) 0이 되는 것 방지지
## 분모에 전체 횟수 더하기 유니크한 토큰의 수 더해주기 

예를들어 긍정일 때, this라는 단어가 등장할 확률을 계산해보겠습니다.

$$P(\text{"this"}|pos)=\frac{2+1}{17+14}=0.0967$$

부정일 때, this라는 단어가 등장할 확률은 아래와 같습니다.

$$P(\text{"this"}|neg)=\frac{1+1}{18+16}=0.08$$

리뷰에 단어들이 등장하는 사건을 모두 독립이라고 가정하면, 긍정일 때 특정 리뷰가 등장할 확률과 부정일 때, 특정 리뷰가 등장할 확률을 계산할 수 있습니다.

$$P(d|pos)=P(\text{"this"}|pos)*P(\text{"is"}|pos)*...=7.69*10^{-9}$$

부정일 때도 마찬가지로 확률을 계산해주면 됩니다.

$$P(d|neg)=P(\text{"this"}|neg)*P(\text{"is"}|neg)*...=1.43*10^{-10}$$

In [17]:
## 수식을 그대로 코드로 적어봄

In [16]:
tokens = tokenizer.tokenize(data[0][0].lower())
tokens = ['this', 'movie', 'was', 'awesome', '!', 'amazing']

pos_total_prob = 1
neg_total_prob = 1
for token in tokens:
    pos_prob = (pos_counter[token] + 1) / (pos_total_cnt + pos_vocabs)
    neg_prob = (neg_counter[token] + 1) / (neg_total_cnt + neg_vocabs)
    pos_total_prob *= pos_prob
    neg_total_prob *= neg_prob
print(f"P(review|pos): {pos_total_prob}")
print(f"P(review|neg): {neg_total_prob}")

P(review|pos): 8.112642408296669e-08
P(review|neg): 7.767973651364808e-09


### 사후 확률 계산

이제 각 조건부 확률을 계산할 수 있으니, 사후 확률을 계산해보겠습니다. 리뷰가 주어졌을 때, 긍정일 확률을 베이즈 정리를 이용하여 표현하면 아래와 같습니다.


$$P(pos|d)=\frac{P(d|pos)P(pos)}{P(d)}=\frac{P(d|pos)P(pos)}{P(d|pos)P(pos)+P(d|neg)P(neg)}$$

In [19]:
pos_prior = 0.5  # data에 긍정 2개 부정 2개였기 때문에 0.5
neg_prior = 0.5
pos_posterior = (pos_total_prob * pos_prior) / (pos_total_prob * pos_prior + neg_total_prob * neg_prob)
print(pos_posterior)

0.9943991077865111


베이즈 정리를 이용해서 계산해보면 "this is a materpiece! i really enjoyed"라는 리뷰가 긍정일 확률은 98%였습니다.

In [23]:
tokens = ['this', 'is', 'a', 'masterpiece', '!', 'i', 'really', 'enjoyed']
pos_posteriorr = (pos_total_prob * pos_prior) / ((pos_total_prob * pos_prior) + (neg_total_prob * neg_prior))
print(pos_posteriorr)

0.9126157120206826


In [24]:
### 장점 : O1의 타이만으로도 인퍼런스가능 모델이 가볍고 엄청 빠르다

## Naive Bayes 한계

Naive Bayes의 가정 때문에 문제가 한계가 있습니다. 먼저 토큰의 등장 위치는 중요합니다. 동일한 토큰일 지라도 앞 뒤에 어느 토큰이 오느냐에 따라서 의미가 달라질 수 있습니다. 

또한 각 토큰이 등장하는 사건은 독립 사건이 아닙니다. 앞에 어느 토큰이 오는지에 따라서 뒤에 어떤 토큰이 올지 결정되기도 합니다. 즉, 텍스트 데이터에서 각각의 토큰은 서로 연관이 있습니다.

그럼에도 불구하고 Naive Bayes는 몹시 가볍고 단순하다는 장점이 있습니다. 때문에 스팸 필터, 텍스트 분류 등의 테스크에서 활용되는 알고리즘입니다.

In [25]:
## 현실에서 잘 사용하지 않음, 딥러닝 모델을 더 자주 씀

## 정리

이번 챕터에서는 베이즈 정리를 이용하여 텍스트를 분류하는 naive bayes classifier에 대해서 알아보았습니다. naive bayes는 굉장히 가벼우면서도 상당한 성능을 보여주어 실제로 간단한 테스크들에 사용되는 기법입니다. 또한 베이즈 정리 개념을 복습하기에도 매우 좋습니다. 

다음 챕터에서는 실제 네이버 영화 리뷰 데이터 셋을 가지고 리뷰가 긍정인지 부정인지 분류하는 모델을 학습시켜 보겠습니다.

In [ ]:
## 네이버 검색 쿼리만 보고 검색 결과 없는 쿼리의 트래픽 지워주기  ->  